<!--
# 01_Data_Load_and_Inspect.ipynb
-->
<h1 style="background-color:#4E9A06; color:#ffffff; padding:10px 15px; border-radius:5px; margin-top:1rem; margin-bottom:1rem; text-align:center;">
  00 – Data Load & Inspect
</h1>

---


In this notebook, we will:

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Objectives
</h2>

- **Load** raw survey and climate CSV files into Pandas DataFrames.
- **Standardize** column names according to the project specifications.
- **Merge** on the key column (`obs`) and identify any initial issues 
- (missing values, invalid durations).
- **Save** a merged inspection file for downstream ETL.

> **Context:**  
> We are integrating plant disease survey data with corresponding climate observations.  
> We focus on enforcing a uniform survey duration window (≤ 6 months) to align with short-term weather–disease relationships.

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Inputs
</h2>

**Plant Disease Survey Data** (`data/raw/complete_plant_disease_database.csv`):  

This CSV holds the raw survey records compiled from literature. Columns include:
- **obs**: Unique observation ID (matching key to climate data).
- **NUM_ID**: Internal numeric identifier for each study in the review.
- **short_reference**: Citation shorthand (first author & year).
- **natural_or_ag**: Indicates plant system type:  
  - `"natural"` (wild)  
  - `"agricultural"` (crop/managed).
- **location**: Named location of the survey (e.g., “County X, Country Y”).
- **parasite_taxonomy**: Taxonomic details of the disease agent (as specific as available).
- **host_taxonomy**: Taxonomic details of the host plant (as specific as available).
- **start_date_survey**: Month and year when the survey began (YYYY-MM).
- **duration_months**: Number of consecutive months surveyed.
- **n**: Total number of plants examined in the survey population.
- **infected**: Number of those plants found infected at the time of survey.
- **incidence**: Infection prevalence (`infected / n`, expressed as a proportion).
  

**Climate & Weather Data** (`data/raw/complete_plant_study_climate_data.csv`):  
  Sourced from Kirk et al. (2025) and available on Dryad 
  ([doi:10.5061/dryad.p8cz8wb0h](https://datadryad.org/dataset/doi:10.5061/dryad.p8cz8wb0h#readme)). 

This CSV contains monthly and annual climate variables for each survey observation. Columns include:
- **obs**: Unique observation ID matching the plant disease survey (linking key).
- **bio01**: Annual historical mean temperature (30-year average, °C).
- **bio12**: Annual historical total precipitation (30-year average, mm).
- **tavg**: Monthly historical mean temperature for the survey period (°C).
- **prec**: Monthly historical precipitation for the survey period (mm/month).
- **total_precipitation**: Contemporaneous precipitation (average during survey months, mm/month).
- **temperature_2m**: Contemporaneous 2-meter air temperature (average during survey months, °C).
- **start_date**: Date when the disease survey began (YYYY-MM-DD).
- **end_date**: Date when the disease survey ended (YYYY-MM-DD).



- **Notebook Environment**:  
  Requires Python packages such as Pandas, NumPy, Matplotlib, and Seaborn. 
  Assumes the repository’s folder structure is already established 
  (`data/raw/`, `data/processed/`, `jupyter_notebooks/`, etc.).

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Outputs
</h2>

Merged Inspection Dataset (`data/processed/merged_inspection.csv`)

This CSV combines the two raw sources (climate/weather + disease survey) into one table for inspection and downstream ETL. It is not yet model-ready.

 Columns include:
- **obs**: Observation ID (linking key).
- **NUM_ID**: Study identifier from the survey data.
- **short_reference**: Study citation (first author, year).
- **natural_or_ag**: System type (wild vs. agricultural).
- **location**: Survey location.
- **parasite_taxonomy** and **host_taxonomy**: Combined taxonomic details.
- **start_date_survey**, **duration_months**, **n**, **infected**, **incidence**: Survey details.
- **bio01**, **bio12**, **tavg**, **prec**, **total_precipitation**, **temperature_2m**, **start_date**, **end_date**: All climate and weather variables mapped to the same `obs`.

> **Note:** The `merged_climate_disease_final.csv` file is derived from the two raw files above. It allows seamless linkage between plant disease observations and their corresponding climatic context, facilitating all downstream EDA, modeling, and visualization steps.

- **DataFrame Inspection Results**:  
  - Overview of DataFrame dimensions, column names, and data types.  
  - Summary statistics (mean, median, min/max) for numeric fields.  
  - Counts and locations of any null or missing values.  
 
- **Documentation Snippets**:  
  - Markdown annotations describing dataset provenance, variable definitions, and any anomalies discovered during inspection.  
  
- **(Optional) Intermediate CSV**:  
  - If lightweight cleaning (e.g., dropping exact duplicates) is performed here, the resulting intermediate file (e.g., `data/processed/merged_inspected.csv`) 
  - can be saved for reference in subsequent ETL steps.  

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
Additional Comments
</h2>

- This notebook focuses strictly on validating and understanding raw inputs; no extensive transformations or model-ready datasets are created at this stage.  
- Any observations about missing data or extreme outliers should be forwarded to the ETL notebook (`02_etl_preprocessing.ipynb`) to ensure a consistent cleanup strategy.  
- For future improvements:  
  - Implement automated schema validation (e.g., check expected columns and data types against a predefined template).  
  - Flag rows with values outside plausible ranges (e.g., temperature anomalies beyond ±5 °C) and log them for review.  



**Reference:**  
Kirk, D., Cohen, J. M., Nguyen, V., et al. (2025). *Impacts of Weather Anomalies and Climate on Plant Disease* [Data set]. Dryad. https://doi.org/10.5061/dryad.p8cz8wb0h  

---

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  1. Environment Setup & Imports
</h2>

In [1]:
from pathlib import Path
import sys
import pandas as pd  
import numpy as np   
import os

# Add style_loader path manually
STYLE_DIR = Path.cwd() / "notebook_style"
if str(STYLE_DIR) not in sys.path:
    sys.path.insert(0, str(STYLE_DIR))

from style_loader import load_style
load_style()
print("Style loaded successfully")

Injected custom CSS from: c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\jupyter_notebooks\notebook_style\custom.css
Style loaded successfully


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
2. Change working directory
</h2>

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [3]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\baner\\OneDrive\\Documenti\\data_analytics_AI\\capstone_CI_main\\climacrop_health\\climacrop_health\\jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [4]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [5]:
current_dir = os.getcwd()
current_dir

'c:\\Users\\baner\\OneDrive\\Documenti\\data_analytics_AI\\capstone_CI_main\\climacrop_health\\climacrop_health'

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  3. Define File Paths
</h2>

This project follows a structured directory layout:
- `data/raw/`: Contains original unmodified input files
  - `climate/`: Climate observation data
  - `survey/`: Plant disease survey records
- `data/processed/`: Stores intermediate and cleaned datasets

In [14]:
# Define project root relative to notebook location
PROJECT_ROOT = Path.cwd()
RAW_DIR = PROJECT_ROOT / "data" / "raw"

# Point to specific files
SURVEY_PATH = RAW_DIR / "complete_plant_disease_database.csv"
CLIMATE_PATH = RAW_DIR / "complete_plant_study_climate_data.csv"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
INSPECT_FILE = PROCESSED_DIR / "merged_inspection.csv"

# Verify paths exist
print(f"Project root exists: {PROJECT_ROOT.exists()}")
print(f"Raw directory exists: {RAW_DIR.exists()}")
print(f"Survey file exists: {SURVEY_PATH.exists()}")
print(f"Climate file exists: {CLIMATE_PATH.exists()}")
print(f"Processed directory exists: {PROCESSED_DIR.exists()}")

# Create the processed directory if it doesn’t exist
RAW_DIR.mkdir(parents=True, exist_ok=True)
PROCESSED_DIR.mkdir(parents=True, exist_ok=True)


print("Output will be saved to:", INSPECT_FILE)


Project root exists: True
Raw directory exists: True
Survey file exists: True
Climate file exists: True
Processed directory exists: True
Output will be saved to: c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\data\processed\merged_inspection.csv


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  4. Load Raw Data
</h2>

In [15]:
# 0.2 Load raw data

# Step 1: Define paths to the raw data files
SURVEY_PATH = RAW_DIR / "complete_plant_disease_database.csv"
CLIMATE_PATH = RAW_DIR / "complete_plant_study_climate_data.csv"

# Step 2: Load the CSVs into DataFrames
survey_df = pd.read_csv(SURVEY_PATH)
climate_df = pd.read_csv(CLIMATE_PATH)

print(f"Loaded `survey_df` with shape {survey_df.shape}")
print(f"Loaded `climate_df` with shape {climate_df.shape}")

Loaded `survey_df` with shape (5906, 44)
Loaded `climate_df` with shape (5906, 12)


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  5. Initial Inspection: Shape & Schema
</h2>

In [16]:
# 5.1) Preview the first few rows of each DataFrame
display(survey_df.head(3))
display(climate_df.head(3))


,NUM_ID,First_author,Study_Year,Person_extracting,Date_extracted,Paper_double_checked,Short reference,Host.species,Host_family,Host_order,...,Start Date (yyyymm),Span (months),Number_sampled_locations,Coarse_spatial_scale,Approx_spatial_accuracy_width_km,Response metric,n,Infected,Incidence,Obs
0,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,28,0.0,0.0,1
1,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,10,0.0,0.0,2
2,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,15,0.0,0.0,3


,system:index,Location,obs,bio01,bio12,end_date,prec,start_date,tavg,temperature_2m,total_precipitation,.geo
0,0,Areas surrounding the Jardin Alpin du Lautaret...,1,95.853316,943.493736,2008-08-31T00:00:00,78.747861,2008-08-01T00:00:00,181.284576,288.956314,0.005280,"{""type"":""Polygon"",""coordinates"":[[[6.028999805..."
1,1,Areas surrounding the Jardin Alpin du Lautaret...,2,50.894780,1283.165287,2008-08-31T00:00:00,96.436140,2008-08-01T00:00:00,133.753340,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.117000102..."
2,2,Areas surrounding the Jardin Alpin du Lautaret...,3,46.951686,1317.325816,2008-08-31T00:00:00,98.158494,2008-08-01T00:00:00,129.520853,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.122000217..."


In [17]:
# 5.2) Print column names and data types
print("--- Survey DataFrame Schema ---")
print(survey_df.info())

print("--- Climate DataFrame Schema ---")
print(climate_df.info())


--- Survey DataFrame Schema ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5906 entries, 0 to 5905
Data columns (total 44 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   NUM_ID                            5906 non-null   object 
 1   First_author                      5906 non-null   object 
 2   Study_Year                        5906 non-null   int64  
 3   Person_extracting                 5906 non-null   object 
 4   Date_extracted                    5906 non-null   object 
 5   Paper_double_checked              5906 non-null   object 
 6   Short reference                   5906 non-null   object 
 7   Host.species                      5864 non-null   object 
 8   Host_family                       5882 non-null   object 
 9   Host_order                        5882 non-null   object 
 10  Host_strain                       725 non-null    object 
 11  Host_type                         590

**Insight:**  
 - Confirm that the obs column exists in both DataFrames and that data types align with expectations.

 - Identify any missing values or unexpected columns early to avoid downstream errors.

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  6. Standardize column names
</h2>

In [18]:
# 6.1) Rename columns in survey_df to match project specification
survey_df.rename(
    columns={
        "Obs": "obs",
        "NUM_ID": "study_id",
        "Short reference": "study_reference",
        "Natural_or_ag": "system_type",
        "Location": "location",
        "n": "n_plants",
        "Infected": "n_infected",
        "Incidence": "incidence",
    },
    inplace=True,
)

climate_df.rename(
    columns={
        "bio01": "annual_mean_temp",
        "bio12": "annual_precip",
        "tavg": "monthly_temp",
        "prec": "monthly_precip",
        "total_precipitation": "contemp_precip",
        "temperature_2m": "contemp_temp",
    },
    inplace=True,
)

# Verify renaming
print("Survey columns now:", survey_df.columns.tolist()[:10])
print("Climate columns now:", climate_df.columns.tolist()[:10])

Survey columns now: ['study_id', 'First_author', 'Study_Year', 'Person_extracting', 'Date_extracted', 'Paper_double_checked', 'study_reference', 'Host.species', 'Host_family', 'Host_order']
Climate columns now: ['system:index', 'Location', 'obs', 'annual_mean_temp', 'annual_precip', 'end_date', 'monthly_precip', 'start_date', 'monthly_temp', 'contemp_temp']


In [19]:
# Preview first few rows of both datasets
print("\nSurvey sample:")
display(survey_df.head())

print("\nClimate sample:")
display(climate_df.head())


Survey sample:


,study_id,First_author,Study_Year,Person_extracting,Date_extracted,Paper_double_checked,study_reference,Host.species,Host_family,Host_order,...,Start Date (yyyymm),Span (months),Number_sampled_locations,Coarse_spatial_scale,Approx_spatial_accuracy_width_km,Response metric,n_plants,n_infected,incidence,obs
0,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,28,0.0,0.000000,1
1,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,10,0.0,0.000000,2
2,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,15,0.0,0.000000,3
3,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,72,2.0,0.027778,4
4,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,200808,1,1.0,Plot,1,Incidence,30,0.0,0.000000,5



Climate sample:


,system:index,Location,obs,annual_mean_temp,annual_precip,end_date,monthly_precip,start_date,monthly_temp,contemp_temp,contemp_precip,.geo
0,0,Areas surrounding the Jardin Alpin du Lautaret...,1,95.853316,943.493736,2008-08-31T00:00:00,78.747861,2008-08-01T00:00:00,181.284576,288.956314,0.005280,"{""type"":""Polygon"",""coordinates"":[[[6.028999805..."
1,1,Areas surrounding the Jardin Alpin du Lautaret...,2,50.894780,1283.165287,2008-08-31T00:00:00,96.436140,2008-08-01T00:00:00,133.753340,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.117000102..."
2,2,Areas surrounding the Jardin Alpin du Lautaret...,3,46.951686,1317.325816,2008-08-31T00:00:00,98.158494,2008-08-01T00:00:00,129.520853,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.122000217..."
3,3,Areas surrounding the Jardin Alpin du Lautaret...,4,30.425471,1458.278376,2008-08-31T00:00:00,106.305506,2008-08-01T00:00:00,112.943493,288.128423,0.005225,"{""type"":""Polygon"",""coordinates"":[[[6.147999763..."
4,4,Areas surrounding the Jardin Alpin du Lautaret...,5,102.943483,898.312969,2008-08-31T00:00:00,77.168637,2008-08-01T00:00:00,189.015197,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.083000183..."


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  7. Merge Survey & Climate Data
</h2>

In [20]:
# 7.1) Confirm merge key presence
if "obs" in survey_df.columns and "obs" in climate_df.columns:
    print("'obs' key found in both datasets. Proceeding to merge.")
else:
    raise KeyError("Merge key 'obs' not found in both datasets")


'obs' key found in both datasets. Proceeding to merge.


Merge on `obs`

In [21]:
# 7.2) Merge datasets on 'obs'
merged_df = pd.merge(survey_df, climate_df, on="obs", how="inner")
print("Merged dataset shape:", merged_df.shape)


Merged dataset shape: (5906, 55)


In [22]:
# Preview merged dataset
print("\nMerged dataset preview:")
display(merged_df.head())



Merged dataset preview:


,study_id,First_author,Study_Year,Person_extracting,Date_extracted,Paper_double_checked,study_reference,Host.species,Host_family,Host_order,...,Location,annual_mean_temp,annual_precip,end_date,monthly_precip,start_date,monthly_temp,contemp_temp,contemp_precip,.geo
0,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,Areas surrounding the Jardin Alpin du Lautaret...,95.853316,943.493736,2008-08-31T00:00:00,78.747861,2008-08-01T00:00:00,181.284576,288.956314,0.005280,"{""type"":""Polygon"",""coordinates"":[[[6.028999805..."
1,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,Areas surrounding the Jardin Alpin du Lautaret...,50.894780,1283.165287,2008-08-31T00:00:00,96.436140,2008-08-01T00:00:00,133.753340,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.117000102..."
2,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,Areas surrounding the Jardin Alpin du Lautaret...,46.951686,1317.325816,2008-08-31T00:00:00,98.158494,2008-08-01T00:00:00,129.520853,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.122000217..."
3,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,Areas surrounding the Jardin Alpin du Lautaret...,30.425471,1458.278376,2008-08-31T00:00:00,106.305506,2008-08-01T00:00:00,112.943493,288.128423,0.005225,"{""type"":""Polygon"",""coordinates"":[[[6.147999763..."
4,715,Abbate,2014,Vianda,2021-06-30,Devin,Abbate_2014,Silene vulgaris,Caryophyllaceae,Caryophyllales,...,Areas surrounding the Jardin Alpin du Lautaret...,102.943483,898.312969,2008-08-31T00:00:00,77.168637,2008-08-01T00:00:00,189.015197,288.499283,0.005288,"{""type"":""Polygon"",""coordinates"":[[[6.083000183..."


> **Insight:**  
> An inner merge ensures we only keep records present in both datasets.  
> If the shape is smaller than expected, investigate unmatched keys.


<h3 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  7.1 Post-Merge validation
</h3>

1) Verify no unexpected data loss from merge operation

In [23]:
print(f"Rows in disease table (before merge): {len(survey_df)}")
print(f"Rows in climate table (before merge): {len(climate_df)}")
print(f"Rows in merged table: {len(merged_df)}")

Rows in disease table (before merge): 5906
Rows in climate table (before merge): 5906
Rows in merged table: 5906


2) Confirm key columns maintained their properties

In [24]:
# 2a) incidence remains between 0 and 1
if "incidence" in merged_df.columns:
    invalid_incidence = merged_df[
        (merged_df["incidence"] < 0)
        | (merged_df["incidence"] > 1)
        | (merged_df["incidence"].isna())
    ]
    print(
        f"Total rows with incidence outside [0,1] or missing: {len(invalid_incidence)}"
    )
    if len(invalid_incidence) > 0:
        display(invalid_incidence.head())

Total rows with incidence outside [0,1] or missing: 0


In [25]:
# 2b) dates preserved formatting (and are properly parsed as datetime)
# First, check the dtype
if "date" in merged_df.columns:
    print("dtype of 'date' column:", merged_df["date"].dtype)
    # If it's object, try to coerce
    if not np.issubdtype(merged_df["date"].dtype, np.datetime64):
        try:
            merged_df["date"] = pd.to_datetime(merged_df["date"], errors="raise")
            print("Converted 'date' to datetime successfully.")
        except Exception as e:
            print("Error converting 'date' to datetime:", e)
    # Optionally, check for any parsing issues:
    bad_dates = merged_df["date"].isna().sum()
    print(f"Number of missing / unparseable dates after coercion: {bad_dates}")

In [26]:
# 2c) no duplicate observations

# For example, if (zone, date) should be unique:
if all(col in merged_df.columns for col in ["zone", "date"]):
    dup_keys = merged_df.duplicated(subset=["zone", "date"], keep=False)
    n_dup = dup_keys.sum()
    print(f"Number of duplicate (zone, date) rows: {n_dup}")
    if n_dup > 0:
        display(merged_df[dup_keys].sort_values(["zone", "date"]).head())

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  8. Missing Values Analysis
</h2>

We analyze missingness in two contexts:
1. **Critical Variables**: Fields essential for climate-disease analysis
   - Disease metrics (incidence, n_plants, n_infected)
   - Climate measurements (temperature, precipitation)
2. **Supplementary Fields**: Additional metadata that may inform but aren't critical
   - Host/pathogen taxonomy
   - Study metadata

In [27]:
# 8.1) Calculate missing values per column
missing_counts = merged_df.isna().sum().sort_values(ascending=False)
missing_pct = (missing_counts / merged_df.shape[0] * 100).round(2)

missing_summary = pd.DataFrame(
    {"missing_count": missing_counts, "missing_pct": missing_pct}
)

print("--- Missing Values Summary (Top 10 Columns) ---")
display(missing_summary.head(10))


--- Missing Values Summary (Top 10 Columns) ---


,missing_count,missing_pct
Water,5851,99.07
Antagonist_isolate,5777,97.82
Host_age,5368,90.89
Host_strain,5181,87.72
Vector_species,2815,47.66
Vector_type,2579,43.67
Transmission_mode,1442,24.42
monthly_temp,525,8.89
monthly_precip,525,8.89
Number_sampled_locations,190,3.22


- Any column with missing values will require treatment (imputation or removal) in the ETL pipeline.

- Early identification of missingness helps plan subsequent cleaning steps.

In [28]:
survey_df.isnull().sum().sort_values(ascending=False).head(10)
climate_df.isnull().sum().sort_values(ascending=False).head(10)

monthly_precip      525
monthly_temp        525
contemp_temp         43
contemp_precip       43
annual_mean_temp      3
annual_precip         3
system:index          0
Location              0
obs                   0
end_date              0
dtype: int64

> **Insight:**  
> High-null survey columns (e.g., `Water`) are likely not useful and will
> be dropped. Moderate missing in climate (e.g., `monthly_temp`) suggests
> imputation will be needed downstream.


### Missing Values to Clean Downstream in ETL

From the inspection above, the following columns require cleaning in the ETL pipeline (`scripts/etl.py` & `02_etl_preprocessing.ipynb`):

- **Climate Data**  
  - `annual_mean_temp` & `annual_precip` (3 missing each): → median imputation  
  - `contemp_temp` & `contemp_precip` (43 missing each): → median imputation  
  - `monthly_temp` & `monthly_precip` (525 missing each, ~9%): → median imputation (or advanced imputer)

- **Survey Data**  
  - `Water`, `Antagonist_isolate`, `Host_age`, `Host_strain` (80–98% missing): likely dropped  
  - Other high-null columns (`Vector_species`, `Vector_type`, `Transmission_mode`): drop or flag if biologically critical

> **Note:** Actual imputation and dropping of these columns will be handled in the ETL stage to keep this notebook focused on inspection.

Those columns are part of the literature‐review metadata in the plant disease survey, but because they’re so sparsely populated they won’t be useful in our climate–disease analysis.

<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
9. Data Validation & Sanity Checks
</h2>



<h3 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
9.1 Incidence Validation
</h3>

In Kirk et al. (2025), the authors explicitly restricted their analysis to studies lasting six months or less. They justify this cutoff on biological and methodological grounds:

 **Biological Relevance**

- Short surveys (≤ 6 months) capture discrete disease‐development cycles tied closely to contemporaneous weather conditions.

- Longer studies risk averaging over multiple seasons or successional phases, diluting the direct impact of anomalies on incidence.

 **Methodological Consistency**

- A uniform duration window ensures comparability across diverse host–pathogen systems and geographies.

- It minimizes confounding by long‐term climatic trends or management changes that might accrue in studies lasting more than half a year .

- Therefore, for our ETL pipeline (Notebook 01), we will enforce the same filter to mirror the paper’s design and maintain that tight linkage between short‐term weather anomalies and disease incidence.


In [29]:
# 9.1) Sanity check on incidence calculation
merged_df["calculated_incidence"] = merged_df["n_infected"] / merged_df["n_plants"]

# Compare with reported incidence
incidence_diff = (merged_df["incidence"] - merged_df["calculated_incidence"]).abs()

# Flag mismatches beyond a tiny threshold (e.g., 1%)
invalid_rows = merged_df[incidence_diff > 0.01]

print(f"Mismatched incidence entries: {len(invalid_rows)}")
display(
    invalid_rows[["n_plants", "n_infected", "incidence", "calculated_incidence"]].head()
)

Mismatched incidence entries: 81


,n_plants,n_infected,incidence,calculated_incidence
264,30,27.0,0.91,0.900000
378,10,0.0,0.02,0.000000
392,13,2.0,0.08,0.153846
395,13,2.0,0.08,0.153846
410,17,2.0,0.06,0.117647


<h3 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
9.2 Duration Constraints
</h3>

In Kirk et al. (2025), the authors explicitly restricted their analysis to studies lasting six months or less. They justify this cutoff on biological and methodological grounds:

 **Biological Relevance**

- Short surveys (≤ 6 months) capture discrete disease‐development cycles tied closely to contemporaneous weather conditions.

- Longer studies risk averaging over multiple seasons or successional phases, diluting the direct impact of anomalies on incidence.

 **Methodological Consistency**

- A uniform duration window ensures comparability across diverse host–pathogen systems and geographies.

- It minimizes confounding by long‐term climatic trends or management changes that might accrue in studies lasting more than half a year.

- Therefore, for our ETL pipeline (Notebook 01), we will enforce the same filter to mirror the paper’s design and maintain that tight linkage between short‐term weather anomalies and disease incidence.


In [30]:
# 9.2) Compute & Flag Invalid Durations
# Convert to datetime
merged_df["start_date"] = pd.to_datetime(merged_df["start_date"], errors="coerce")
merged_df["end_date"] = pd.to_datetime(merged_df["end_date"], errors="coerce")

# Compute duration in months
merged_df["duration_mo"] = (
    (merged_df["end_date"].dt.year - merged_df["start_date"].dt.year) * 12
    + (merged_df["end_date"].dt.month - merged_df["start_date"].dt.month)
    + 1
)

# Identify invalid durations (≤0 or >6 months, or NaN)
invalid_duration_rows = merged_df[
    merged_df["duration_mo"].isna()
    | (merged_df["duration_mo"] <= 0)
    | (merged_df["duration_mo"] > 6)
]
print("Invalid durations (will drop in ETL):", len(invalid_duration_rows))


Invalid durations (will drop in ETL): 0


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  10. Save Inspection CSV
</h2>

In [31]:
# 10.1) Write the merged inspection DataFrame to CSV
merged_df.to_csv(INSPECT_FILE, index=False)
print("Inspection file written to:", INSPECT_FILE)

Inspection file written to: c:\Users\baner\OneDrive\Documenti\data_analytics_AI\capstone_CI_main\climacrop_health\climacrop_health\data\processed\merged_inspection.csv


<h2 style="color: #4E9A06; margin-top: 1rem; margin-bottom: 0.5rem;">
  11. Conclusion & Next Steps
</h2>

- We have successfully loaded, standardized, and merged the plant disease survey data with climate observations.  
- Any missing or invalid values were flagged; duplicates were checked.  
- The merged dataset is now saved under `data/processed/merged_inspection.csv`.  
- **Next:** Proceed to the ETL notebook to clean/transform features and prepare for modeling.
